Loading the necessary libraries

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import pytesseract
import time

Defining the braille shapes for each letter/number/space

In [ ]:
# Braille dictionary mapping each letter to its Braille representation
braille_dict = {
    'a': '100000',
    'b': '101000',
    'c': '110000',
    'd': '110100',
    'e': '100100',
    'f': '111000',
    'g': '111100',
    'h': '101100',
    'i': '011000',
    'j': '011100',
    'k': '100010',
    'l': '101010',
    'm': '110010',
    'n': '110110',
    'o': '100110',
    'p': '111010',
    'q': '111110',
    'r': '101110',
    's': '011010',
    't': '011110',
    'u': '100011',
    'v': '101011',
    'w': '011101',
    'x': '110011',
    'y': '110111',
    'z': '100111',
    ' ': '000000',
    '#': '010111',
    '0': '011100',
    '1': '100000',
    '2': '101000',
    '3': '110000',
    '4': '110100',
    '5': '100100',
    '6': '111000',
    '7': '111100',
    '8': '101100',
    '9': '011000'
}

Function converting text to braille

In [ ]:
def convert_to_braille(text):
    braille_text = ''
    for char in text:
        # Use the indicator for numbers if the character is a digit
        if char.isdigit():
            braille_text += braille_dict['#'] + ' ' + braille_dict.get(char.lower(), char)+ ' '
            #print(braille_text)
        else:
            braille_text += braille_dict.get(char.lower(), char) + ' '
            #print(braille_text)
    return braille_text

Function outputting the braille (converted text)

In [ ]:

def show_braille_from_text(input_text):
    braille_text = convert_to_braille(input_text)
    z=1
    for braille_char in braille_text.split():
        if z==9:
            z=1
            time.sleep(5)
        display_braille(braille_char,z)
        time.sleep(0.1)
       #print(z)
        z += 1

In [ ]:
# def show_braille_from_text(input_text):
#     braille_text = convert_to_braille(input_text)
#     length=int(len(braille_text)/7)
#     print(length)

#     for i in range(0, len(braille_text), 50):
#         display_braille(braille_text[i:i+50])
#         time.sleep(0.5)


Function outputting the braille (converted image)

In [ ]:
def show_braille_from_camera():
    # cap = cv2.VideoCapture(0)  # Use the default camera (you may need to change this)
    frame=cv2.imread('/Users/kanjisugimori/Documents/VS_Code/Book.png')
    while True:
        # ret, frame = cap.read()
        # if not ret:
        #     break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        text = pytesseract.image_to_string(gray)
        cv2.imshow("Camera", frame)
        
        print(text)

        if text:
            show_braille_from_text(text)
            
        cv2.destroyAllWindows()
        break

    # cap.release()
    

In [ ]:
def display_braille(braille_pattern,z):
    # Create a blank image
    global global_img

    # Create a blank image if it's the first iteration
    if z == 1 or global_img is None:
        global_img = Image.new('1', (1200, 160), color=255)

    # Get the pixels of the global image
    pixels = global_img.load()

    # Fill the image based on Braille pattern
    for i in range(3):
        for j in range(2):
            index = i * 2 + j
            if index < len(braille_pattern) and braille_pattern[index] == '1':
                for x in range(40):
                    for y in range(40):
                        if (x - 20) ** 2 + (y - 20) ** 2 <= 400:
                            pixels[j * 60 + x +150*(z-1), i * 60 + y] = 0

    # Convert the image to PhotoImage format for displaying in Tkinter
    img_tk = ImageTk.PhotoImage(global_img)
    braille_label.config(image=img_tk)
    braille_label.image = img_tk
    root.update()

In [ ]:
# Create the main window
root = tk.Tk()
root.title("Braille Converter")

# Entry widget for user input
entry = tk.Entry(root, width=30)
entry.pack(pady=10)

# Button to trigger conversion from text
convert_text_button = tk.Button(root, text="Convert from Text", command=lambda: show_braille_from_text(entry.get()))
convert_text_button.pack(pady=10)

# Button to trigger conversion from camera
convert_camera_button = tk.Button(root, text="Convert from Camera", command=show_braille_from_camera)
convert_camera_button.pack(pady=10)

# Label to display Braille patterns
braille_label = tk.Label(root, image=None)
braille_label.pack(pady=10)

# Run the main loop
root.mainloop()